### Installing pyERGM

In [1]:
# -q to hide all installation logs but feel free to remove it.
!pip3 install git+https://github.com/mallaham/pyERGM -q

In [ ]:
!pwd # can be used to know your local directory

In [2]:
import pyergm # to ensure all installation was successful

### Importing pyERGM components

In [3]:
from pyergm import rpy_interface as rpyInterface
from pyergm.network_statistics import NetworkStats
from pyergm.ergm import pyERGM, ModelDiagnostics
from pyergm.simulator import Simulator
from pyergm.data_transformer import DataTransformer
import pandas as pd
from rpy2.robjects import NA_Real

NumExpr defaulting to 4 threads.


### Setting up R environment

In [4]:
rpackages = ['statnet', 'ergm','texreg', 'base', 'grDevices']
renv = rpyInterface.intializeRenv(mirror=1, r_packages= rpackages)
installed_packages = renv.setup_renv()

Installing R packages...
Importing package: statnet
Importing package: ergm
Importing package: texreg
Importing package: base
Importing package: grDevices


### Loading and processing data

In [5]:
buyInFromYouEdgelist = pd.read_csv("buyInFromYouEdgelist.csv")

dt = DataTransformer(renv)
department = dt.rdf_to_list(dt.to_rdf(pd.read_csv("departmentNode.csv")), "department")
leader = dt.rdf_to_list(dt.to_rdf(pd.read_csv("leaderNode.csv")),"leader")
tenure = dt.rdf_to_list(dt.to_rdf(pd.read_csv("tenureNode.csv")), "tenure")
office = dt.rdf_to_list(dt.to_rdf(pd.read_csv("officeNode.csv")), "office")
female = dt.rdf_to_list(dt.to_rdf(pd.read_csv("femaleNode.csv")), "female")
messageEdgeList = pd.read_csv("messageEdgelist.csv")

# converting pandas dataframe to r
r_buyInFromYouEdgelist = dt.to_rdf(buyInFromYouEdgelist)

# loading edgelist as a matrix
buyIn = dt.edgelist_to_matrix(r_buyInFromYouEdgelist)

# set network attributes
buyIn = dt.set_vertex_attribute(buyIn, ["department","leader","tenure","office","female"], [department, leader, tenure, office, female])
# alternative method
# buyIn = dt.set_vertex_attribute(buyIn, "department", department) # Categorical variable for department
# buyIn = dt.set_vertex_attribute(buyIn, "leader", leader) # Indicator variable for department leader
# buyIn = dt.set_vertex_attribute(buyIn, "tenure",tenure) # Years tenure
# buyIn = dt.set_vertex_attribute(buyIn, "office", office) # Indicator variable for whether they are located in the main or secondary office
# buyIn = dt.set_vertex_attribute(buyIn, "female",female) # Indicator


Converting pandas DataFrame to R dataframe...
Converting R dataframe to R list...
Converting pandas DataFrame to R dataframe...
Converting R dataframe to R list...
Converting pandas DataFrame to R dataframe...
Converting R dataframe to R list...
Converting pandas DataFrame to R dataframe...
Converting R dataframe to R list...
Converting pandas DataFrame to R dataframe...
Converting R dataframe to R list...
Converting pandas DataFrame to R dataframe...
Generating a matrix from edgelist...
A list of attribute names and values was passed...
Setting vertex attribute to department...
Setting vertex attribute to leader...
Setting vertex attribute to tenure...
Setting vertex attribute to office...
Setting vertex attribute to female...


### Creating covariance matrix

In [6]:
cov_matrix = dt.cov_matrix(messageEdgeList, 'SenderId',' ReceiverId', 66, 66, NA_Real)

# filling matrix with values 
for index, row in messageEdgeList.iterrows():
    s_idx = int(row['SenderId'])
    r_idx = int(row['ReceiverId'])
    cov_matrix.rx[renv.load_robject('cbind')(s_idx,r_idx)] = float(row['MessagesSent']/100)


Generating covariance matrix for: None
Importing package: base
No covariance column was passed. Returning a matrix with populated values of NA_real_


### Calculating network metrics

In [7]:
################################
## Calculate network metrics
################################
ns = NetworkStats(renv)
summary = ns.summary(buyIn)
print(summary)
network_size = ns.network_size(buyIn)
print(network_size)
betweeness = ns.betweeness(buyIn)
print(betweeness)
isolates = ns.isolates(buyIn)
print(isolates)

Calculating network summary statistics...
Network attributes:
  vertices = 66
  directed = TRUE
  hyper = FALSE
  loops = FALSE
  multiple = FALSE
  bipartite = FALSE
 total edges = 225 
   missing edges = 0 
   non-missing edges = 225 
 density = 0.05244755 

Vertex attributes:

 department:
   character valued attribute
   attribute summary:
   BE    ME     O     P     S Sales    TI    WB    WF 
   10    18     2    13     5     1     9     1     7 

 female:
   integer valued attribute
   66 values

 leader:
   integer valued attribute
   66 values

 office:
   integer valued attribute
   66 values

 tenure:
   integer valued attribute
   66 values
  vertex.names:
   character valued attribute
   66 valid vertex names

No edge attributes

Network edgelist matrix:
       [,1] [,2]
  [1,]    1   31
  [2,]    1   57
  [3,]    2   31
  [4,]    2   59
  [5,]    3   24
  [6,]    3   33
  [7,]    3   52
  [8,]    4   14
  [9,]    4   17
 [10,]    4   23
 [11,]    4   63
 [12,]    4   65
 [

### ERGM model

In [8]:
# ERGM model
# formula and input variables
formula = "buyIn ~ edges + mutual + edgecov(hundreds_messages) + nodemix('leader',base = 3)"
vars = {"buyIn": buyIn, "hundred_messages": cov_matrix, "hundreds_messages": cov_matrix}

# model definition and parameters
ergm = pyERGM(installed_packages['ergm'], formula, vars, constraints="~bd(maxout=5)")
params = dict({"formula":ergm.formula, "constraints":ergm.constraints}) # make sure to include = for parameters that require = vs parameters that can be upacked such as formula
model = ergm.fit_model(params)
summary = ergm.summary(model)
print(summary)

Model summary before fitting ERGM...
In term ‘nodemix’ in package ‘ergm’: Argument ‘base’ has been superseded by ‘levels2’, and it is recommended to use the latter.  Note that its interpretation may be different.
                           c(edges = 225, mutual = 12, edgecov.hundreds_messages = 56.69, mix.leader.0.0 = 94, mix.leader.1.0 = 8, mix.leader.1.1 = 12)
edges                                                                 225.00                                                                           
mutual                                                                 12.00                                                                           
edgecov.hundreds_messages                                              56.69                                                                           
mix.leader.0.0                                                         94.00                                                                           
mix.leader.1.0             

### Model diagnostics

In [9]:
#####################
## Model Diagnositics
#####################
ergm_diagnostics = ModelDiagnostics(renv)

# MCMC
filename, mcmc_results = ergm_diagnostics.run_mcmc(model)
print(filename)

Running MCMC diagnostics with seed 321...
Sample statistics summary:

Iterations = 118784:2353152
Thinning interval = 2048 
Number of chains = 1 
Sample size per chain = 1092 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

                              Mean     SD Naive SE Time-series SE
edges                      0.07692 10.425  0.31547         0.3925
mutual                    -0.14377  2.953  0.08937         0.1107
edgecov.hundreds_messages -0.72039 10.754  0.32543         0.5053
mix.leader.0.0            -0.71520  8.036  0.24319         0.2815
mix.leader.1.0             0.25733  2.461  0.07447         0.0787
mix.leader.1.1            -0.38370  2.839  0.08592         0.1641

2. Quantiles for each variable:

                            2.5%    25%   50%   75% 97.5%
edges                     -19.00 -7.000  0.00 7.000 20.00
mutual                     -5.00 -2.000  0.00 2.000  6.00
edgecov.hundreds_messages -20.65 -8.318 -0.78 6.262 20.4

### Goodness of fit test

In [10]:
## Goodness of fit
gof_params = dict({"verbose=":"T","burnin=":1e+5,"interval=":1e+5})
gof = ergm_diagnostics.gof(model, gof_params, n=100)
print(gof)

Running goodness of fit test...
Importing package: grDevices
Goodness of fit report can be accessed in the following path: ./gof_report.png
Function executed in 7.1200 seconds

Goodness-of-fit for in-degree 

          obs min  mean max MC p-value
idegree0   29   4 11.36  18       0.00
idegree1   17  12 18.44  28       0.94
idegree2    4   8 15.74  26       0.00
idegree3    1   4  9.19  15       0.00
idegree4    4   0  3.60   8       0.96
idegree5    1   0  1.18   5       1.00
idegree6    1   0  0.39   3       0.66
idegree7    0   0  0.07   1       1.00
idegree8    0   0  0.03   1       1.00
idegree9    1   0  0.00   0       0.00
idegree10   0   0  0.01   1       1.00
idegree11   1   0  0.07   1       0.14
idegree12   1   0  0.08   1       0.16
idegree13   0   0  0.09   1       1.00
idegree14   1   0  0.17   2       0.30
idegree15   0   0  0.22   2       1.00
idegree16   2   0  0.35   3       0.14
idegree17   0   0  0.42   2       1.00
idegree18   0   0  0.41   3       1.00
idegree19  

### Simulating networks

In [11]:
###############
## Simulations
###############
sim_parms = {"burnin=":100000, "interval=":100000, "verbose=":"T", "seed=":10}
simulator = Simulator(model, sim_parms)
sim_results = simulator.simulate()
print(len(simulator.get_triangles()))

Simulating 1000 networks with seed vale 340...
Counting the number of triangles in the simulated networks...
1000
